In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import math

# Step 1: Load the Dataset
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Split the dataset into training and test sets
X_train, X_new, y_train, y_new = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Initialize base learners
model_1 = BayesianRidge()
model_2 = DecisionTreeRegressor(random_state=42)
model_3 = SVR()
base_learners = [model_1, model_2, model_3]

# Step 3: Assign equal prior probabilities
priors = np.array([1/3, 1/3, 1/3])

# Step 4: Compute BIC scores for each model
def compute_bic(model, X, y):
    model.fit(X, y)
    y_pred = model.predict(X)
    # Number of parameters estimation
    if hasattr(model, 'coef_'):
        n_params = len(model.coef_)
    elif isinstance(model, DecisionTreeRegressor):
        n_params = model.tree_.node_count
    else:
        n_params = len(model.support_)
    mse = mean_squared_error(y, y_pred)
    n = len(y)
    bic = n * np.log(mse) + n_params * np.log(n)
    return bic

bic_scores = np.array([compute_bic(model, X_train, y_train) for model in base_learners])

# Step 5: Compute posterior probabilities
log_marginal_likelihoods = -0.5 * bic_scores
log_unnormalized_posteriors = log_marginal_likelihoods + np.log(priors)
log_posterior_probabilities = log_unnormalized_posteriors - np.logaddexp.reduce(log_unnormalized_posteriors)
posterior_probabilities = np.exp(log_posterior_probabilities)

print("Posterior Probabilities:")
for i, prob in enumerate(posterior_probabilities):
    print(f"Model {i+1}: {prob:.4f}")

# Step 6: Make predictions on new data
predictions = np.array([model.predict(X_new) for model in base_learners])

# Step 7: Compute BMA prediction
bma_prediction = np.dot(posterior_probabilities, predictions)

# Evaluate the BMA prediction
mse_bma = mean_squared_error(y_new, bma_prediction)
print(f"\nBMA Prediction MSE: {mse_bma:.4f}")

# Compare with individual models
for i, model in enumerate(base_learners):
    mse_model = mean_squared_error(y_new, predictions[i])
    print(f"Model {i+1} MSE: {mse_model:.4f}")

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Step 1: Load and Prepare the Dataset
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
X_train, X_new, y_train, y_new = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 2: Define BoTorch Models as Base Learners
import torch
from botorch.models import SingleTaskGP
from gpytorch.kernels import ScaleKernel, RBFKernel, MaternKernel, LinearKernel
from gpytorch.mlls import ExactMarginalLogLikelihood

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)

model_1 = SingleTaskGP(
    X_train_tensor,
    y_train_tensor,
    covar_module=ScaleKernel(RBFKernel()),
)
model_2 = SingleTaskGP(
    X_train_tensor,
    y_train_tensor,
    covar_module=ScaleKernel(MaternKernel(nu=2.5)),
)
model_3 = SingleTaskGP(
    X_train_tensor,
    y_train_tensor,
    covar_module=ScaleKernel(LinearKernel()),
)
base_learners = [model_1, model_2, model_3]

# Step 3: Assign Prior Probabilities
priors = np.array([1/3, 1/3, 1/3])

# Step 4: Compute Marginal Likelihoods
def compute_log_marginal_likelihood(model, X, y):
    model.train()
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    training_iterations = 50
    for _ in range(training_iterations):
        optimizer.zero_grad()
        output = model(X)
        loss = -mll(output, y).sum()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        log_marg_likelihood = mll(model(X), y).item()
    return log_marg_likelihood

log_marginal_likelihoods = np.array([
    compute_log_marginal_likelihood(model, X_train_tensor, y_train_tensor)
    for model in base_learners
])

# Step 5: Compute Posterior Model Probabilities
log_priors = np.log(priors)
log_unnormalized_posteriors = log_marginal_likelihoods + log_priors
max_log_post = np.max(log_unnormalized_posteriors)
log_unnormalized_posteriors -= max_log_post  # For numerical stability
unnormalized_posteriors = np.exp(log_unnormalized_posteriors)
posterior_probabilities = unnormalized_posteriors / np.sum(unnormalized_posteriors)

print("Posterior Probabilities:")
for i, prob in enumerate(posterior_probabilities):
    print(f"Model {i+1}: {prob:.4f}")

# Step 6: Make Predictions
X_new_tensor = torch.tensor(X_new.values, dtype=torch.float32)

def predict(model, X):
    model.eval()
    with torch.no_grad():
        posterior = model.posterior(X)
        mean = posterior.mean.squeeze(-1)
        variance = posterior.variance.squeeze(-1)
    return mean.numpy(), variance.numpy()

predictions_mean = []
predictions_variance = []

for model in base_learners:
    mean, variance = predict(model, X_new_tensor)
    predictions_mean.append(mean)
    predictions_variance.append(variance)

predictions_mean = np.array(predictions_mean)
predictions_variance = np.array(predictions_variance)

# Step 7: Compute the Bayesian Model Averaged Prediction
bma_mean = np.dot(posterior_probabilities, predictions_mean)
bma_variance = (
    np.dot(posterior_probabilities, predictions_variance) +
    np.dot(posterior_probabilities, (predictions_mean - bma_mean[None, :]) ** 2)
)

# Step 8: Evaluate the Models
from sklearn.metrics import mean_squared_error

mse_bma = mean_squared_error(y_new, bma_mean)
print(f"\nBMA Prediction MSE: {mse_bma:.4f}")

for i, mean in enumerate(predictions_mean):
    mse_model = mean_squared_error(y_new, mean)
    print(f"Model {i+1} MSE: {mse_model:.4f}")

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import math

# Step 1: Load and Prepare the Dataset
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
X_train, X_new, y_train, y_new = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 2: Define scikit-learn Models as Base Learners
model_1 = BayesianRidge()
model_2 = DecisionTreeRegressor(random_state=42)
model_3 = SVR()
model_4 = RandomForestRegressor(random_state=42)
base_learners = [model_1, model_2, model_3, model_4]

# Step 3: Assign Prior Probabilities
priors = np.array([1/len(base_learners)] * len(base_learners))

# Step 4: Compute BIC Scores
def compute_bic(model, X, y):
    model.fit(X, y)
    y_pred = model.predict(X)
    # Estimate the number of parameters
    if hasattr(model, 'coef_'):
        n_params = len(model.coef_)
    elif hasattr(model, 'feature_importances_'):
        n_params = np.count_nonzero(model.feature_importances_)
    elif hasattr(model, 'n_support_'):
        n_params = model.n_support_.sum()
    else:
        n_params = X.shape[1]  # Fallback to number of features
    mse = mean_squared_error(y, y_pred)
    n = len(y)
    bic = n * np.log(mse) + n_params * np.log(n)
    return bic

bic_scores = np.array([compute_bic(model, X_train, y_train) for model in base_learners])

# Step 5: Compute Posterior Probabilities
log_marginal_likelihoods = -0.5 * bic_scores
log_priors = np.log(priors)
log_unnormalized_posteriors = log_marginal_likelihoods + log_priors
max_log_post = np.max(log_unnormalized_posteriors)
log_unnormalized_posteriors -= max_log_post  # For numerical stability
unnormalized_posteriors = np.exp(log_unnormalized_posteriors)
posterior_probabilities = unnormalized_posteriors / np.sum(unnormalized_posteriors)

print("Posterior Probabilities:")
for i, prob in enumerate(posterior_probabilities):
    print(f"Model {i+1}: {prob:.4f}")

# Step 6: Make Predictions
predictions = np.array([model.predict(X_new) for model in base_learners])

# Step 7: Compute BMA Prediction
bma_prediction = np.dot(posterior_probabilities, predictions)

# Step 8: Evaluate the Models
mse_bma = mean_squared_error(y_new, bma_prediction)
print(f"\nBMA Prediction MSE: {mse_bma:.4f}")

for i, pred in enumerate(predictions):
    mse_model = mean_squared_error(y_new, pred)
    print(f"Model {i+1} MSE: {mse_model:.4f}")

Posterior Probabilities:
Model 1: 0.0000
Model 2: 1.0000
Model 3: 0.0000
Model 4: 0.0000

BMA Prediction MSE: 0.4952
Model 1 MSE: 0.5556
Model 2 MSE: 0.4952
Model 3 MSE: 1.3320
Model 4 MSE: 0.2554


In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import math

# Step 1: Load and Prepare the Dataset
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
X_train, X_new, y_train, y_new = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 2: Define scikit-learn Models as Base Learners
model_1 = BayesianRidge()
model_2 = DecisionTreeRegressor(random_state=42)
model_3 = SVR()
model_4 = RandomForestRegressor(random_state=42)
base_learners = [model_1, model_2, model_3, model_4]

# Step 3: Assign Prior Probabilities
priors = np.array([1/len(base_learners)] * len(base_learners))

# Step 4: Compute BIC Scores
def compute_bic(model, X, y):
    model.fit(X, y)
    y_pred = model.predict(X)
    # Estimate the number of parameters
    if hasattr(model, 'coef_'):
        n_params = len(model.coef_)
    elif hasattr(model, 'feature_importances_'):
        n_params = np.count_nonzero(model.feature_importances_)
    elif hasattr(model, 'n_support_'):
        n_params = model.n_support_.sum()
    else:
        n_params = X.shape[1]  # Fallback to number of features
    mse = mean_squared_error(y, y_pred)
    n = len(y)
    bic = n * np.log(mse) + n_params * np.log(n)
    return bic, n_params

# Step 5: Collect BIC Scores and Number of Parameters
bic_scores = []
n_params_list = []

for model in base_learners:
    bic, n_params = compute_bic(model, X_train, y_train)
    bic_scores.append(bic)
    n_params_list.append(n_params)

bic_scores = np.array(bic_scores)

# Step 6: Compute Posterior Probabilities
log_marginal_likelihoods = -0.5 * bic_scores
log_priors = np.log(priors)
log_unnormalized_posteriors = log_marginal_likelihoods + log_priors
max_log_post = np.max(log_unnormalized_posteriors)
log_unnormalized_posteriors -= max_log_post  # For numerical stability
unnormalized_posteriors = np.exp(log_unnormalized_posteriors)
posterior_probabilities = unnormalized_posteriors / np.sum(unnormalized_posteriors)

# Step 7: Print BIC Scores, Number of Parameters, and Posterior Probabilities
print("BIC Scores and Number of Parameters:")
for i, (bic, n_params) in enumerate(zip(bic_scores, n_params_list)):
    print(f"Model {i+1}: BIC = {bic:.4f}, Number of Parameters = {n_params}")

print("\nPosterior Probabilities:")
for i, prob in enumerate(posterior_probabilities):
    print(f"Model {i+1}: {prob:.4f}")

# Step 8: Make Predictions
predictions = np.array([model.predict(X_new) for model in base_learners])

# Step 9: Compute BMA Prediction
bma_prediction = np.dot(posterior_probabilities, predictions)

# Step 10: Evaluate the Models
mse_bma = mean_squared_error(y_new, bma_prediction)
print(f"\nBMA Prediction MSE: {mse_bma:.4f}")

for i, pred in enumerate(predictions):
    mse_model = mean_squared_error(y_new, pred)
    print(f"Model {i+1} MSE: {mse_model:.4f}")


BIC Scores and Number of Parameters:
Model 1: BIC = -10785.6666, Number of Parameters = 8
Model 2: BIC = -1177970.9512, Number of Parameters = 8
Model 3: BIC = 152304.7942, Number of Parameters = 15149
Model 4: BIC = -55123.4526, Number of Parameters = 8

Posterior Probabilities:
Model 1: 0.0000
Model 2: 1.0000
Model 3: 0.0000
Model 4: 0.0000

BMA Prediction MSE: 0.4952
Model 1 MSE: 0.5556
Model 2 MSE: 0.4952
Model 3 MSE: 1.3320
Model 4 MSE: 0.2554
